In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from StringIO import StringIO

In [ ]:
parcsv = sc.textFile("/home/milroy/pyspark/2014-04-24.csv", 24)

segs = parcsv.map(lambda l: l.split(";"))

#rdd = segs.map(lambda p: (str(p[0]), str(p[1]), str(p[2]), str(p[3]), str(p[4]), int(p[5].strip())))

rdd = segs.map(lambda p: (str(p[1]), str(p[2]), int(p[5].strip())))

schema_struct = StructType([StructField('metric', StringType(), True),
                     StructField('value', StringType(), True),
                     StructField('timestamp', IntegerType(), True)])

df = sqlCtx.createDataFrame(rdd, schema_struct)

def seqPivot(u, v):
    if not u:
        u = {}
    u[v.metric] = v.value
    return u

def cmbPivot(u1, u2):
    u1.update(u2)
    return u1

pivot = (
    df
    .rdd
    .keyBy(lambda row: row.timestamp)
    .aggregateByKey(None, seqPivot, cmbPivot)
)
columns = (
    pivot
    .values()
    .map(lambda u: set(u.keys()))
    .reduce(lambda s,t: s.union(t))
)
result = sqlCtx.createDataFrame(
    pivot
    .map(lambda (k, u): [k] + [u.get(c) for c in columns]),
    schema=StructType(
        [StructField('timestamp', IntegerType())] + 
        [StructField(c, StringType()) for c in columns]
    )
)
result.show()

In [28]:
import pandas

In [29]:
p_df = result.toPandas()

In [31]:
p_df

,timestamp,lustre.scratch.ost.obdfilter.OST0015.process_config,lustre.scratch.ost.obdfilter.OST0002.bytesfree,lustre.scratch.ost.obdfilter.OST0017.setattr,lustre.scratch.ost.obdfilter.OST0010.llog_connect,lustre.scratch.ost.obdfilter.OST0005.getattr,diskstat_sdah_percent_io_time,lustre.scratch.ost.obdfilter.OST0030.process_config,lustre.scratch.ost.obdfilter.OST0034.cache_hit,diskstat_sdr_io_time,...,lustre.scratch.ost.obdfilter.OST0021.get_info,lustre.scratch.ost.obdfilter.OST0028.commitrw,diskstat_sdh_percent_io_time,lustre.scratch.ost.obdfilter.OST0023.get_info,diskstat_sdag_reads_merged,lustre.scratch.ost.obdfilter.OST0014.destroy,diskstat_sdad_write_time,lustre.scratch.ost.obdfilter.OST0027.commitrw,diskstat_sdbg_read_time,diskstat_sdz_write_bytes_per_sec
0,1398398976,0.00000,4835605217280.00000,0.00000,0.00000,-0.19995,0.000000,0.00000,None,0.000000,...,0.00000,0.000000,0.00000,0.000000,None,0.00000,0.000000,0.00000,0.000000,0.000000
1,1398382596,0.00000,None,0.39995,None,0.00000,0.000000,None,None,0.000000,...,None,0.000000,None,0.000000,None,None,0.000000,0.79991,0.000000,0.000000
2,1398392838,0.00000,None,0.59947,None,0.00000,0.000000,None,None,0.000000,...,None,0.000000,None,0.000000,None,None,0.000000,0.59945,0.000000,0.000000
3,1398383118,None,0.00000,None,0.00000,None,0.000000,0.00000,None,0.000000,...,0.54254,0.000000,0.00000,0.000000,None,0.18085,0.000000,None,0.000000,0.000000
4,1398393360,None,None,None,None,None,None,None,0.00000,None,...,None,None,0.39863,None,0.39840,None,None,None,None,None
5,1398400344,None,4834276147200.00000,None,0.00000,None,0.000000,0.00000,None,0.000000,...,0.00000,0.000000,0.19856,0.000000,None,0.00000,0.000000,None,0.000000,0.000000
6,1398398994,None,0.00000,None,0.00000,None,0.246547,0.00000,0.00000,0.000000,...,27.81789,7.569673,0.39953,0.000000,0.00000,0.00000,0.000000,None,0.000000,0.000000
7,1398401082,None,0.00000,None,0.00000,None,0.223329,0.00000,0.00000,0.000000,...,7.40993,0.133331,None,0.000000,0.00000,0.11578,0.000000,None,0.000000,0.000000
8,1398387222,None,4849790353408.00000,None,0.00000,None,0.003332,0.00000,0.00000,0.000000,...,0.00000,0.063300,None,0.000000,0.00000,0.00000,0.000000,None,0.000000,0.000000
9,1398392856,0.00000,None,0.39987,None,0.00000,3.139113,None,0.00000,0.000000,...,None,0.056651,0.00000,0.000000,0.00000,None,0.000000,0.00000,0.000000,0.000000
